In [1]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import datetime
from time import sleep

from lyricsgenius import Genius
from collections import Counter

import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
import requests
import pprint
from spotipy.oauth2 import SpotifyClientCredentials

from requests.exceptions import ReadTimeout

In [2]:
CLIENT_ID = '7f8b2e3adcbf4c24a6091bbaf44013de'
CLIENT_SECRET = '71e10a4bb2534a2cb97e19b794ebc38f'


AUTH_URL = 'https://accounts.spotify.com/api/token'

# POST
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
})

# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token
access_token = auth_response_data['access_token']

In [3]:
auth_manager = SpotifyClientCredentials(CLIENT_ID, CLIENT_SECRET)
sp = spotipy.Spotify(auth_manager=auth_manager, requests_timeout=10, retries=10)

In [4]:
genius = Genius('in424DYZRH1oa--Iyzce_Ue6uLYNqt1Q6-Yq20PUSYl-PjvMNuuaX0YBiwcSFt9_')

In [5]:
song_parts_regex = r"\[Intro.*?\]|\[Verse [0-9].*?\]|\[Refrain.*?\]|\[Pre-Chorus.*?\]|\[Chorus.*?\]|\[Post-Chorus.*?\]|\[Hooks.*?\]|\[Riffs/Basslines.*?\]|\[Scratches.*?\]|\[Bridge.*?\]|\[Interlude.*?\]|\[Break.*?\]|\[Skit.*?\]|\[Collision.*?\]|\[Instrumental or Solo.*?\]|\[Ad lib.*?\]|\[Segue.*?\]|\[Outro.*?\]"

In [6]:
def scrape_lyrics(url):
    """
    input (str): url for a Genius lyrics page
    output (tuple list): returns list of song parts and lyrics tuples
    """

    if type(url) != str or len(url) <= 3:
        return []

    page = requests.get(url)
    html = BeautifulSoup(page.text, 'html.parser')
    lyrics_result_set = html.find_all('div', attrs={'data-lyrics-container': 'true'})
    lyrics = [lyrics_result.get_text(separator=' ') for lyrics_result in lyrics_result_set]
    lyrics = ' '.join(lyrics)
   
    try:
        parts_iter = re.finditer(song_parts_regex, lyrics)

        song_parts = []
        song_part_spans = []
        for part_object in parts_iter:
            song_parts.append(part_object.group())
            song_part_spans.append(part_object.span())

        lyric_spans = []
        # # If there are lyrics before the first song part, add that range of indices
        # try:
        #     print(song_part_spans[0][0])
        # except:
        #     print("An exception occurred", song_part_spans)
        # first_song_part_start_index = song_part_spans[0][0]
        # if first_song_part_start_index != 0:
        #     song_parts.insert(0, '[Start]')
        #     lyric_spans.append((0, first_song_part_start_index))

        for index in range(len(song_part_spans) - 1):
            start_lyric_span = song_part_spans[index][1]
            end_lyric_span = song_part_spans[index + 1][0]
            lyric_spans.append((start_lyric_span, end_lyric_span))

        # Add the lyrics after the final song part
        lyric_spans.append((song_part_spans[-1][1], len(lyrics)))

        # Get lyrics without song part tags
        span_to_lyrics = lambda span: lyrics[span[0]:span[1]].lower()
        lyrics_without_song_parts = [span_to_lyrics(span).strip() for span in lyric_spans]

        # Create lyrics list
        lyrics = list(zip(song_parts, lyrics_without_song_parts))

        return lyrics
        
    except:
        return []


def extract_lyrics_text(lyrics):
        text = []
        if lyrics:
            # print()
            # print(lyrics)
            # print()
            # print(list(zip(*lyrics)))
            # print()
            # print(list(zip(*lyrics))[1])
            # print()
            # print(' '.join(list(zip(*lyrics))[1]))
            # print()
            # print(list(' '.join(list(zip(*lyrics))[1])))
            return ' '.join(list(zip(*lyrics))[1])

In [7]:
df_charts = pd.read_csv('data/charts.csv')
df_charts.head()

,date,rank,song,artist,last-week,peak-rank,weeks-on-board
0,2021-11-06,1,Easy On Me,Adele,1.0,1,3
1,2021-11-06,2,Stay,The Kid LAROI & Justin Bieber,2.0,1,16
2,2021-11-06,3,Industry Baby,Lil Nas X & Jack Harlow,3.0,1,14
3,2021-11-06,4,Fancy Like,Walker Hayes,4.0,3,19
4,2021-11-06,5,Bad Habits,Ed Sheeran,5.0,2,18


In [11]:
def create_df_billboard(start=0, limit=5, chunk=1000):
    df_charts = pd.read_csv('data/charts.csv')
    song_artist_array = df_charts[['song', 'artist']].to_numpy()
    # print(song_artist_array[:limit])
    # print([song_info for (song_info, _) in Counter(map(tuple, song_artist_array)).most_common()[:limit]])
    songs, artists = list(zip(*[song_info for (song_info, _) in Counter(map(tuple, song_artist_array)).most_common()[start:limit]]))

    urls = []
    peak_ranks = []
    weeks_on_boards = []
    last_weeks = []
    ranks = []
    lyrics_temp = []
    lyrics_text_temp = []
    genres = []
    decades = []
    features = []
    releases = []
    durations = []
    tempos = []
    timesigs = []

    for indx, (song, artist) in enumerate(zip(songs, artists)):
        
        # Spotify/genre work
        q = "artist:" + artist + " track:" + song

        try:
            results = sp.search(q=q, type='track')
        except ReadTimeout:
            print('Spotify track timed out... trying one more time...')
            try:
                results = sp.search(q=q, type='track')
            except ReadTimeout:
                print('Spotify track timed out twice.')
                results = {'tracks' : {'items' : []}}
                # results['tracks']['items'] = []
        # results = sp.search(q=q, type='track')
        if len(results['tracks']['items']) > 0:

            # artist genre
            artist_uri = results['tracks']['items'][0]['artists'][0]['uri']
            # artist_spot = sp.artist(artist_uri)
            try:
                artist_spot = sp.artist(artist_uri)
                genre = artist_spot['genres']
            except ReadTimeout:
                print('Spotify artist timed out... trying one more time...')
                try:
                    artist_spot = sp.artist(artist_uri)
                    genre = artist_spot['genres']
                except ReadTimeout:
                    print('Spotify artist timed out twice.')
                    genre = None
            # genre = artist_spot['genres']
            genres.append(genre)

            # release and decade
            release = results['tracks']['items'][0]['album']['release_date']
            releases.append(release)
            decade = ((int(release[:4]))//10) * 10
            decades.append(decade)

            # song features
            spot_id = results['tracks']['items'][0]['id']
            try:
                feature = sp.audio_features([spot_id])[0]

                features.append(feature)
                # song duration
                durations.append(feature['duration_ms'])
                # song tempo
                tempos.append(feature['tempo'])
                # song time signature
                timesigs.append(feature['time_signature'])
            except ReadTimeout:
                print('Spotify features timed out... trying one more time...')
                try:
                    feature = sp.audio_features([spot_id])[0]
                    features.append(feature)
                    # song duration
                    durations.append(feature['duration_ms'])
                    # song tempo
                    tempos.append(feature['tempo'])
                    # song time signature
                    timesigs.append(feature['time_signature'])
                except ReadTimeout:
                    print('Spotify features timed out twice.')
                    features.append(None)
                    # song duration
                    durations.append(None)
                    # song tempo
                    tempos.append(None)
                    # song time signature
                    timesigs.append(None)
        else:
            genres.append(None)
            releases.append(None)
            decades.append(None)
            features.append(None)
            durations.append(None)
            tempos.append(None)
            timesigs.append(None)

        # Genius/lyric work
        start = datetime.datetime.now()
        stop = start + datetime.timedelta(seconds = 5)
        while True:
            if datetime.datetime.now() > stop:
                urls.append(None)
                break
            try:
                urls.append(genius.search_song(song, artist).url)
                break
            except:
                pass
        match_rows = df_charts.loc[(df_charts['song'] == song) & (df_charts['artist'] == artist)]
        peak_rank = {}
        weeks_on_board = {}
        last_week = {}
        rank = {}
        for index, row in match_rows.iterrows():
            rank[row['date']] = row['rank']
            peak_rank[row['date']] = row['peak-rank']
            weeks_on_board[row['date']] = row['weeks-on-board']
            last_week[row['date']] = row['last-week']
        ranks.append(rank)
        peak_ranks.append(peak_rank)
        weeks_on_boards.append(weeks_on_board)
        last_weeks.append(last_week)

        if ((indx) % chunk) == 0 and indx != 0:
            if indx == chunk:
                lyrics_temp = lyrics_temp + list(map(scrape_lyrics, urls[0:]))
                lyrics_text_temp = lyrics_text_temp + list(map(extract_lyrics_text, lyrics_temp))
                # lyrics_temp.append(list(map(scrape_lyrics, urls[0:])))
                # lyrics_text_temp.append(list(map(extract_lyrics_text, lyrics_temp)))
            else:
                lyrics_temp = lyrics_temp + list(map(scrape_lyrics, urls[(indx - chunk + 1):]))
                # lyrics_temp.append(list(map(scrape_lyrics, urls[(indx - chunk + 1):])))
                lyrics_text_temp = lyrics_text_temp + list(map(extract_lyrics_text, lyrics_temp[(indx - chunk + 1):]))
                # lyrics_text_temp.append(list(map(extract_lyrics_text, lyrics_temp[(indx - chunk + 1):])))

            # print(len(songs))
            # print(len(artists))
            # print(len(urls))
            # print(len(lyrics_temp))
            # print(len(lyrics_text_temp))
            # print(len(decades))
            # print(len(timesigs))
            df_billboard_temp = pd.DataFrame({
            'song': songs[:(indx + 1)],
            'artist': artists[:(indx + 1)],
            'url': urls,
            'lyrics': lyrics_temp,
            'lyrics_text': lyrics_text_temp,
            'rank': ranks,
            'peak_rank': peak_ranks,
            'weeks_on_board': weeks_on_boards,
            'last_week': last_weeks,
            'genre': genres,
            'decade': decades,
            'release_date': releases,
            'song_features': features,
            'duration_ms': durations,
            'tempo': tempos,
            'time_signature': timesigs,
            })

            df_billboard_temp.to_csv(('data_scrape/chart_with_lyrics_genres_running' + '.csv'), index=False)
            print('wrote: ' + 'chart_with_lyrics_genres_' + str(indx) + '.csv')
        
        sleep(0.6)

    lyrics = list(map(scrape_lyrics, urls))
    lyrics_text = list(map(extract_lyrics_text, lyrics))

    df_billboard = pd.DataFrame({
        'song': songs,
        'artist': artists,
        'url': urls,
        'lyrics': lyrics_temp,
        'lyrics_text': lyrics_text_temp,
        'rank': ranks,
        'peak_rank': peak_ranks,
        'weeks_on_board': weeks_on_boards,
        'last_week': last_weeks,
        'genre': genres,
        'decade': decades,
        'release_date': releases,
        'song_features': features,
        'duration_ms': durations,
        'tempo': tempos,
        'time_signature': timesigs,
    })

    return df_billboard
    

In [13]:
df_billboard = create_df_billboard(start=0, limit=25000, chunk=10)

Searching for "Blinding Lights" by The Weeknd...
Searching for "Radioactive" by Imagine Dragons...
Done.
Searching for "Sail" by AWOLNATION...
Done.
Searching for "I'm Yours" by Jason Mraz...
Done.
Searching for "How Do I Live" by LeAnn Rimes...
Done.
Searching for "Counting Stars" by OneRepublic...
Done.
Searching for "Party Rock Anthem" by LMFAO Featuring Lauren Bennett & GoonRock...
Done.
Searching for "Rolling In The Deep" by Adele...
Done.
Searching for "Foolish Games/You Were Meant For Me" by Jewel...
Done.
Searching for "Before He Cheats" by Carrie Underwood...
Searching for "I Hope" by Gabby Barrett Featuring Charlie Puth...
Done.
wrote: chart_with_lyrics_genres_10.csv
Searching for "Ho Hey" by The Lumineers...
Done.
Searching for "You And Me" by Lifehouse...
Done.
Searching for "Circles" by Post Malone...
Searching for "Demons" by Imagine Dragons...
Done.
Searching for "Need You Now" by Lady Antebellum...
Done.
Searching for "Macarena (Bayside Boys Mix)" by Los Del Rio...
No r

In [65]:
df_charts = pd.read_csv('data/charts.csv')
song_artist_array = df_charts[['song', 'artist']].to_numpy()
# print(song_artist_array[:limit])
# print([song_info for (song_info, _) in Counter(map(tuple, song_artist_array)).most_common()[:limit]])
songs, artists = list(zip(*[song_info for (song_info, _) in Counter(map(tuple, song_artist_array)).most_common()]))

df_new = pd.DataFrame({'song' : songs})
df_new.to_csv(('test.csv'), index=False)